# Extract, Transform & Load

Import libraries.

In [1]:
import pandas as pd

Load CSV.

In [7]:
df_traffic_accidents = pd.read_csv("../data/raw/atus_anual_csv/conjunto_de_datos/atus_anual_2023.csv")

In [8]:
df_traffic_accidents.info

<bound method DataFrame.info of         COBERTURA  ID_ENTIDAD  ID_MUNICIPIO  ANIO  MES  ID_HORA  ID_MINUTO  \
0       Municipal           1             1  2023    1        0          0   
1       Municipal           1             1  2023    1        0          0   
2       Municipal           1             1  2023    1        2         20   
3       Municipal           1             1  2023    1        3         20   
4       Municipal           1             1  2023    1        6          0   
...           ...         ...           ...   ...  ...      ...        ...   
396623  Municipal          32            56  2023   12       14         45   
396624  Municipal          32            56  2023   12       21         40   
396625  Municipal          32            56  2023   12       14         30   
396626  Municipal          32            56  2023   12        0          0   
396627  Municipal          32            56  2023   12       18          0   

        ID_DIA  DIASEMANA      

In [9]:
df_traffic_accidents.columns

Index(['COBERTURA', 'ID_ENTIDAD', 'ID_MUNICIPIO', 'ANIO', 'MES', 'ID_HORA',
       'ID_MINUTO', 'ID_DIA', 'DIASEMANA', 'URBANA', 'SUBURBANA', 'TIPACCID',
       'AUTOMOVIL', 'CAMPASAJ', 'MICROBUS', 'PASCAMION', 'OMNIBUS', 'TRANVIA',
       'CAMIONETA', 'CAMION', 'TRACTOR', 'FERROCARRI', 'MOTOCICLET',
       'BICICLETA', 'OTROVEHIC', 'CAUSAACCI', 'CAPAROD', 'SEXO', 'ALIENTO',
       'CINTURON', 'ID_EDAD', 'CONDMUERTO', 'CONDHERIDO', 'PASAMUERTO',
       'PASAHERIDO', 'PEATMUERTO', 'PEATHERIDO', 'CICLMUERTO', 'CICLHERIDO',
       'OTROMUERTO', 'OTROHERIDO', 'NEMUERTO', 'NEHERIDO', 'CLASACC',
       'ESTATUS'],
      dtype='object')

In [10]:
df_traffic_accidents.dtypes

COBERTURA       object
ID_ENTIDAD       int64
ID_MUNICIPIO     int64
ANIO             int64
MES              int64
ID_HORA          int64
ID_MINUTO        int64
ID_DIA           int64
DIASEMANA       object
URBANA          object
SUBURBANA       object
TIPACCID        object
AUTOMOVIL        int64
CAMPASAJ         int64
MICROBUS         int64
PASCAMION        int64
OMNIBUS          int64
TRANVIA          int64
CAMIONETA        int64
CAMION           int64
TRACTOR          int64
FERROCARRI       int64
MOTOCICLET       int64
BICICLETA        int64
OTROVEHIC        int64
CAUSAACCI       object
CAPAROD         object
SEXO            object
ALIENTO         object
CINTURON        object
ID_EDAD          int64
CONDMUERTO       int64
CONDHERIDO       int64
PASAMUERTO       int64
PASAHERIDO       int64
PEATMUERTO       int64
PEATHERIDO       int64
CICLMUERTO       int64
CICLHERIDO       int64
OTROMUERTO       int64
OTROHERIDO       int64
NEMUERTO         int64
NEHERIDO         int64
CLASACC    

Create a date column.

In [11]:
df_traffic_accidents["fecha_hora"] = pd.to_datetime(dict(
        year=df_traffic_accidents["ANIO"],
        month=df_traffic_accidents["MES"],
        day=df_traffic_accidents["ID_DIA"],
        hour=df_traffic_accidents["ID_HORA"],
        minute=df_traffic_accidents["ID_MINUTO"]
    ), errors='coerce')

Drop NaT values.

In [12]:
df_traffic_accidents = df_traffic_accidents.dropna()
df_traffic_accidents["fecha_hora"].isna().sum()

np.int64(0)

Drop year, month, day, hour and minute columns.

In [13]:
df_traffic_accidents = df_traffic_accidents.drop(["ANIO","MES","ID_DIA","ID_HORA","ID_MINUTO"],axis=1)

In [14]:
df_traffic_accidents.columns

Index(['COBERTURA', 'ID_ENTIDAD', 'ID_MUNICIPIO', 'DIASEMANA', 'URBANA',
       'SUBURBANA', 'TIPACCID', 'AUTOMOVIL', 'CAMPASAJ', 'MICROBUS',
       'PASCAMION', 'OMNIBUS', 'TRANVIA', 'CAMIONETA', 'CAMION', 'TRACTOR',
       'FERROCARRI', 'MOTOCICLET', 'BICICLETA', 'OTROVEHIC', 'CAUSAACCI',
       'CAPAROD', 'SEXO', 'ALIENTO', 'CINTURON', 'ID_EDAD', 'CONDMUERTO',
       'CONDHERIDO', 'PASAMUERTO', 'PASAHERIDO', 'PEATMUERTO', 'PEATHERIDO',
       'CICLMUERTO', 'CICLHERIDO', 'OTROMUERTO', 'OTROHERIDO', 'NEMUERTO',
       'NEHERIDO', 'CLASACC', 'ESTATUS', 'fecha_hora'],
      dtype='object')

Create a zone column.

In [15]:
zones = pd.DataFrame()
zones["URBANA"] = df_traffic_accidents["URBANA"].replace("Sin accidente en esta zona", pd.NA)
zones["SUBURBANA"] = df_traffic_accidents["SUBURBANA"].replace("Sin accidente en esta zona", pd.NA)

df_traffic_accidents["zona"] = zones["URBANA"].combine_first(zones["SUBURBANA"])

Fill rows with undefined zone.

In [16]:
df_traffic_accidents["zona"] = df_traffic_accidents["zona"].fillna('No especificado')

Drop URBANA and SUBURBANA columns.

In [17]:
df_traffic_accidents = df_traffic_accidents.drop(["URBANA","SUBURBANA"],axis=1)

In [18]:
df_traffic_accidents.columns

Index(['COBERTURA', 'ID_ENTIDAD', 'ID_MUNICIPIO', 'DIASEMANA', 'TIPACCID',
       'AUTOMOVIL', 'CAMPASAJ', 'MICROBUS', 'PASCAMION', 'OMNIBUS', 'TRANVIA',
       'CAMIONETA', 'CAMION', 'TRACTOR', 'FERROCARRI', 'MOTOCICLET',
       'BICICLETA', 'OTROVEHIC', 'CAUSAACCI', 'CAPAROD', 'SEXO', 'ALIENTO',
       'CINTURON', 'ID_EDAD', 'CONDMUERTO', 'CONDHERIDO', 'PASAMUERTO',
       'PASAHERIDO', 'PEATMUERTO', 'PEATHERIDO', 'CICLMUERTO', 'CICLHERIDO',
       'OTROMUERTO', 'OTROHERIDO', 'NEMUERTO', 'NEHERIDO', 'CLASACC',
       'ESTATUS', 'fecha_hora', 'zona'],
      dtype='object')

Change columns name.

In [20]:
rename_columns = {
    "ID_ENTIDAD": "entidad_id",
    "ID_MUNICIPIO": "municipio_id",
    "TIPACCID": "tipo_accidente",
    "AUTOMOVIL": "automovil",
    "CAMPASAJ": "camioneta_pasajeros",
    "MICROBUS": "microbus",
    "PASCAMION": "camion_urbano",
    "OMNIBUS": "autobus",
    "TRANVIA": "tranvia",
    "CAMIONETA": "camioneta_carga",
    "CAMION": "camion_carga",
    "TRACTOR": "tractor",
    "FERROCARRI": "ferrocarril",
    "MOTOCICLET": "motocicleta",
    "BICICLETA": "bicicleta",
    "OTROVEHIC": "otro_vehiculo",
    "CAUSAACCI": "causa_accidente",
    "CAPAROD": "superficie_rodamiento",
    "SEXO": "sexo",
    "ALIENTO": "aliento_alcoholico",
    "CINTURON": "cinturon",
    "ID_EDAD": "edad",
    "CONDMUERTO":"conductor_muerto",
    "CONDHERIDO": "conductor_herido",
    "PASAMUERTO": "pasajero_muerto",
    "PASAHERIDO": "pasajero_herido",
    "PEATMUERTO": "peaton_muerto",
    "PEATHERIDO": "peaton_herido",
    "CICLMUERTO": "ciclista_muerto",
    "CICLHERIDO": "ciclista_herido",
    "OTROMUERTO": "otro_muerto",
    "OTROHERIDO": "otro_herido",
    "NEMUERTO": "no_espec_muerto",
    "NEHERIDO": "no_espec_herido"
}

for original, new in rename_columns.items():
    if original in df_traffic_accidents.columns:
        df_traffic_accidents.rename(columns={original: new}, inplace=True)

In [21]:
df_traffic_accidents.columns

Index(['COBERTURA', 'entidad_id', 'municipio_id', 'DIASEMANA',
       'tipo_accidente', 'automovil', 'camioneta_pasajeros', 'microbus',
       'camion_urbano', 'autobus', 'tranvia', 'camioneta_carga',
       'camion_carga', 'tractor', 'ferrocarril', 'motocicleta', 'bicicleta',
       'otro_vehiculo', 'causa_accidente', 'superficie_rodamiento', 'sexo',
       'aliento_alcoholico', 'cinturon', 'edad', 'conductor_muerto',
       'conductor_herido', 'pasajero_muerto', 'pasajero_herido',
       'peaton_muerto', 'peaton_herido', 'ciclista_muerto', 'ciclista_herido',
       'otro_muerto', 'otro_herido', 'no_espec_muerto', 'no_espec_herido',
       'CLASACC', 'ESTATUS', 'fecha_hora', 'zona'],
      dtype='object')

Drop unnecessary columns.

In [22]:
df_traffic_accidents = df_traffic_accidents.drop(['COBERTURA', 'DIASEMANA', 'CLASACC', 'ESTATUS'], axis=1)
df_traffic_accidents.columns

Index(['entidad_id', 'municipio_id', 'tipo_accidente', 'automovil',
       'camioneta_pasajeros', 'microbus', 'camion_urbano', 'autobus',
       'tranvia', 'camioneta_carga', 'camion_carga', 'tractor', 'ferrocarril',
       'motocicleta', 'bicicleta', 'otro_vehiculo', 'causa_accidente',
       'superficie_rodamiento', 'sexo', 'aliento_alcoholico', 'cinturon',
       'edad', 'conductor_muerto', 'conductor_herido', 'pasajero_muerto',
       'pasajero_herido', 'peaton_muerto', 'peaton_herido', 'ciclista_muerto',
       'ciclista_herido', 'otro_muerto', 'otro_herido', 'no_espec_muerto',
       'no_espec_herido', 'fecha_hora', 'zona'],
      dtype='object')